# Single frequency component modelling of speech HMGM

This notebook will examine the plausibility of modelling the real and imaginary part of warped frequency coefficients using an auto-regressive model.

This notebook comprises the following steps:
1. Load data
2. Pass data through warped filter bank and calculate warped frequency coefficients
3. Model the warped frequency coefficients using an auto-regressive model
4. Infer auto-regressive parameters
5. Evaluate performance on coefficient prediction

## Load packages and functions

In [1]:
using ForneyLab       # based on last official release
using LinearAlgebra
using Plots
using PyPlot
using ProgressMeter
using Compat
using WAV
using Base64
using FileIO
using HDF5
using FLAC
using DSP
using LAR
using LAR.AR
using LAR.DataAR
using FFTW

include("../functions/auxiliary/buffer.jl")
include("../functions/auxiliary/audioplayer.jl")
include("../functions/auxiliary/workflow.jl")
include("../functions/auxiliary/fourier.jl")
include("../functions/auxiliary/warpedfilter.jl")
;

┌ Info: Precompiling ForneyLab [9fc3f58a-c2cc-5bff-9419-6a294fefdca9]
└ @ Base loading.jl:1273
ERROR: LoadError: LoadError: could not open file C:\Users\s151781\.julia\packages\ForneyLab\Hz4kD\src\engines\julia\update_rules\dot_product.jl
Stacktrace:
 [1] include at .\boot.jl:328 [inlined]
 [2] include_relative(::Module, ::String) at .\loading.jl:1105
 [3] include at .\Base.jl:31 [inlined]
 [4] include(::String) at C:\Users\s151781\.julia\packages\ForneyLab\Hz4kD\src\ForneyLab.jl:1
 [5] top-level scope at C:\Users\s151781\.julia\packages\ForneyLab\Hz4kD\src\engines\julia\julia.jl:26
 [6] include at .\boot.jl:328 [inlined]
 [7] include_relative(::Module, ::String) at .\loading.jl:1105
 [8] include at .\Base.jl:31 [inlined]
 [9] include(::String) at C:\Users\s151781\.julia\packages\ForneyLab\Hz4kD\src\ForneyLab.jl:1
 [10] top-level scope at C:\Users\s151781\.julia\packages\ForneyLab\Hz4kD\src\ForneyLab.jl:108
 [11] include at .\boot.jl:328 [inlined]
 [12] include_relative(::Module, ::Str

ErrorException: Failed to precompile ForneyLab [9fc3f58a-c2cc-5bff-9419-6a294fefdca9] to C:\Users\s151781\.julia\compiled\v1.3\ForneyLab\Eacon_DT1kD.ji.

## Adjustable parameters

In [2]:
# signal preprocessing
sound_file = "../data/obama.wav"      # specify sound location
fs_desired = 8000                 # specify desired sampling frequency

# signal warping
nr_taps_warped = 32                # number of taps of warped filter bank
z_pole = 0.4092                    # pole location of all-pass filter segments (0.5756 ~ approximation Bark scale)
window = rectangularwindow         # specify windowing function from fourier.jl file

# inference and modelling
bin = 8                            # specify which bin to model (should be between 1 and half `nr_taps_warped`)
bin_c = "real"                     # specify whether to model the `real` or `imaginary` part
AR_order = 1                       # order of auto-regressive model
iterations = 10                     # number of iterations for variational message passing
Λ_meas = 1e10                      # assumed precision of measurement noise
;

UndefVarError: UndefVarError: rectangularwindow not defined

## Load data

In [3]:
# load and process speech signal
x_time, fs_time = wavread(sound_file, format="native")
fs_time = Int(fs_time)
x_time = x_time[20*fs_time+1:23*fs_time]
x_time = resample(x_time, fs_desired/fs_time)
x_time = 100*x_time/var(x_time)

# specify time axis
t = collect((1:length(x_time))/fs_desired)

# create audioplayer
audioplayers(x_time, fs=fs_desired)
;

UndefVarError: UndefVarError: wavread not defined

## Visualize data

In [4]:
# create figure with subplots
_, ax = plt.subplots(ncols=2, figsize=(15,5))

# plot speech signal as a function of time
ax[1].plot(t, x_time)

# plot spectrogram of speech signal
plot_spectrogram(spectrogram(x_time, convert(Int, 100e-3*fs_desired), convert(Int, 10e-3*fs_desired), fs=fs_desired), fs_desired, ax=ax[2])

# visual modifications
ax[1].grid(), ax[1].set_ylabel("x[t]"), ax[1].set_xlabel("time [sec]"), ax[1].set_title("speech signal"), ax[2].set_title("spectrogram")
;

UndefVarError: UndefVarError: plt not defined

## Warp signal and calculate warped frequency distribution

In [5]:
# initialize memory vector of taps, Y, with zeros
Y = zeros(nr_taps_warped*3,1)

# initialize memory struct for filter taps and spectrum coefficients
mem_taps = Array{Array{Float64,1}}(undef, length(x_time))
mem_spec = Array{Complex{Float64},2}(undef, length(x_time), nr_taps_warped)

# get update matrices as defined in thesis
W, U = allpass_update_matrix(nr_taps_warped, z_pole)

# loopt through time-domain signal
for k = 1:length(x_time)
    
    # update memory vector of taps and taps themselves
    Y, mem_taps[k] = allpass_update(x_time[k], Y, nr_taps_warped, z_pole, T=W, u=U)

    # calculate windowed FFT of taps
    mem_spec[k,:] = FFTW.fft(window(nr_taps_warped).*mem_taps[k])
    
end

# discard frequency components concerning non-positive frequencies (are complex conjugate, because of real temporal signal)
if nr_taps_warped%2 == 0
    mem_spec = hcat(mem_spec[:, 2:Int(nr_taps_warped/2)], conj.(mem_spec[:, Int(nr_taps_warped/2)+1]))
else
    mem_spec = mem_spec[:, 2:Int(ceil(nr_taps_warped/2))]
end

# calculate real and imaginary parts of remaining frequency components
mem_spec_real = real(mem_spec)
mem_spec_imag = imag(mem_spec)
;

UndefVarError: UndefVarError: x_time not defined

## Warped spectral distribution visualization

In [6]:
# calculate power of spectral components
mem_spec_pwr = 20*log10.(abs.(mem_spec))'

# plot warped power spectrum
plt.imshow(mem_spec_pwr, aspect="auto", cmap="jet", origin="lower", extent=[t[1], t[end], 1, Int(floor(nr_taps_warped/2))]), plt.xlabel("time [sec]"), plt.ylabel("frequency bin"), plt.title("warped frequency power distribution")
;

UndefVarError: UndefVarError: mem_spec not defined

## Single bin visualization

In [7]:
# select specified frequency bin
if bin_c == "real"
    spec_select = mem_spec_real[:, bin]
else
    spec_select = mem_spec_imag[:, bin]
end

# create subplots
_, ax = plt.subplots(ncols=3, figsize=(15,5))

# plot frequency component as a function of time
ax[1].plot(t, spec_select)

# plot frequency distribution of frequency component
ax[2].plot(FFTW.fftfreq(length(spec_select), fs_desired), abs.(FFTW.fft(spec_select)))

# plot spectrogram of speech signal
plot_spectrogram(spectrogram(spec_select, convert(Int, 100e-3*fs_desired), convert(Int, 10e-3*fs_desired), fs=fs_desired), fs_desired, ax=ax[3])

# change visuals
ax[1].grid(), ax[1].set_xlabel("time [sec]"), ax[1].set_ylabel("frequency component"), ax[1].set_title("frequency component over time"), ax[2].grid(), ax[2].set_xlabel("frequency [Hz]"), ax[2].set_ylabel("magnitude"), ax[2].set_title("frequency distribution of component"), ax[3].set_title("spectrogram of frequency component")
;

UndefVarError: UndefVarError: bin_c not defined

## Generative model and inference

In [8]:
# create factor graphs
fg = FactorGraph()

# Switching
@RV T ~ ForneyLab.Dirichlet(ones(3,3))
@RV m1 ~ ForneyLab.GaussianMeanPrecision(randn(AR_order), 1e-5*Ic(AR_order))
@RV w1 ~ ForneyLab.Wishart(tiny*diageye(AR_order), AR_order)
@RV m2 ~ ForneyLab.GaussianMeanPrecision(zeros(AR_order), 1e10*Ic(AR_order))
@RV w2 ~ ForneyLab.Wishart(huge*diageye(AR_order), AR_order)
@RV m3 ~ ForneyLab.GaussianMeanPrecision(zeros(AR_order), 1e10*Ic(AR_order))
@RV w3 ~ ForneyLab.Wishart(huge*diageye(AR_order), AR_order)

@RV [id=:zprev] zprev ~ ForneyLab.Categorical(1/3*ones(3))

@RV [id=:z] z ~ ForneyLab.Transition(zprev, T)

# AR node
@RV γ ~ ForneyLab.Gamma(placeholder(:a_γ), placeholder(:b_γ))
#@RV θ ~ GaussianMeanPrecision(placeholder(:μ_θ, dims=(AR_order,)), placeholder(:Λ_θ, dims=(AR_order, AR_order)))
@RV θ ~ GaussianMixture(z, m1, w1, m2, w2, m3, w3)
@RV Sprev ~ GaussianMeanPrecision(placeholder(:μ_Sprev, dims=(AR_order,)), placeholder(:Λ_Sprev, dims=(AR_order, AR_order)))
@RV S ~ LAR.AR.Autoregressive(θ, Sprev, γ)

# selection and input
d = zeros(AR_order)
d[1] = 1
@RV x ~ DotProduct(d, S)
@RV v ~ GaussianMeanPrecision(0, Λ_meas)
@RV y = x + v
placeholder(y, :y)

ForneyLab.draw(fg)
ForneyLab.draw(fg, external_viewer=:default)
;

UndefVarError: UndefVarError: FactorGraph not defined

In [9]:
# specify recognition factorization and create algorithm
q = RecognitionFactorization([zprev; z], T, m1, w1, m2, w2, m3, w3, θ, S, Sprev, γ, ids=[:z :T :m1 :w1 :m2 :w2 :m3 :w3 :θ :S :Sprev :γ])
algo = compatibility_fix(variationalAlgorithm(q))

# evaluate algorith 
eval(Meta.parse(algo))
;

UndefVarError: UndefVarError: zprev not defined

In [10]:
# priors
current_a_γ = 0.001
current_b_γ = 0.001
current_Λ_θ = 1e0*Ic(AR_order) #1e4*diagm(randn(AR_order))
current_μ_θ = randn(AR_order)
current_μ_Sprev = randn(AR_order)
current_Λ_Sprev = 1e1*diagm(randn(AR_order))
F_tot = []
a_γ_tot = []
b_γ_tot = []
μ_S_tot = []
Λ_S_tot = []
μ_θ_tot = []
Λ_θ_tot = []
μ_pred = Float64[]
μ_pred2 = Float64[]
messages=Array{Message}(undef, 6)

p = Progress(length(spec_select), 1, "Observed ")
marginals = Dict()

marginals[:T] = vague(Dirichlet, (3,3))
marginals[:m1] = ProbabilityDistribution(Multivariate, GaussianMeanPrecision, m=randn(AR_order), w=1e-5*Ic(AR_order))
marginals[:w1] = ProbabilityDistribution(Wishart, v=tiny*diageye(AR_order), nu=AR_order)
marginals[:m2] = ProbabilityDistribution(Multivariate, GaussianMeanPrecision, m=zeros(AR_order), w=1e5*Ic(AR_order))
marginals[:w2] = ProbabilityDistribution(Wishart, v=tiny*diageye(AR_order), nu=AR_order)
marginals[:m3] = ProbabilityDistribution(Multivariate, GaussianMeanPrecision, m=zeros(AR_order), w=1e5*Ic(AR_order))
marginals[:w3] = ProbabilityDistribution(Wishart, v=tiny*diageye(AR_order), nu=AR_order)

# loop through signal samples
for sp in 1:length(spec_select)
    
    # update progress bar
    update!(p, sp)
    
    # update marginals
    marginals[:θ] = ProbabilityDistribution(Multivariate, GaussianMeanPrecision, m=current_μ_θ, w=current_Λ_θ)
    marginals[:γ] = ProbabilityDistribution(Univariate, Gamma, a=current_a_γ, b=current_b_γ)
    marginals[:Sprev] = ProbabilityDistribution(Multivariate, GaussianMeanPrecision, m=current_μ_Sprev, w=current_Λ_Sprev)

    # perform VMP iterations
    for it = 1:iterations
        
        # update data dictionary
        data = Dict(:y => spec_select[sp],
                    :μ_Sprev => current_μ_Sprev,
                    :Λ_Sprev => current_Λ_Sprev,
                    :a_γ => current_a_γ,
                    :b_γ => current_b_γ)
        
        #marginals = Base.invokelatest(stepS!, data, marginals, messages)  

        stepz!(data, marginals)
        stepw1!(data, marginals)
        stepw2!(data, marginals)
        stepw3!(data, marginals)
        stepm1!(data, marginals)
        stepm2!(data, marginals)
        stepm3!(data, marginals)
        stepT!(data, marginals)
        stepS!(data, marginals)
        stepSprev!(data, marginals)
        stepθ!(data, marginals)
        stepγ!(data, marginals)
        
        # save new parameters
        push!(a_γ_tot, marginals[:γ].params[:a])
        push!(b_γ_tot, marginals[:γ].params[:b])
        push!(μ_S_tot, ForneyLab.unsafeMean(marginals[:S])[1])
        push!(Λ_S_tot, ForneyLab.unsafePrecision(marginals[:S])[1])
        push!(μ_θ_tot, ForneyLab.unsafeMean(marginals[:θ]))
        push!(Λ_θ_tot, ForneyLab.unsafePrecision(marginals[:θ]))
        
        # update current parameters
        current_μ_θ = ForneyLab.unsafeMean(marginals[:θ])
        current_Λ_θ = ForneyLab.unsafePrecision(marginals[:θ])
        current_a_γ = marginals[:γ].params[:a]
        current_b_γ = marginals[:γ].params[:b]
        current_μ_Sprev = ForneyLab.unsafeMean(marginals[:Sprev])
        current_Λ_Sprev = ForneyLab.unsafePrecision(marginals[:Sprev])
    end
    
    current_μ_Sprev = ForneyLab.unsafeMean(marginals[:S])
    current_Λ_Sprev = ForneyLab.unsafePrecision(marginals[:S])
    
    # perform 1 step prediction
    push!(μ_pred, sum(current_μ_Sprev.*current_μ_θ))
    #push!(μ_pred2, ForneyLab.unsafeMean(messages[3].dist))
    
end
;

UndefVarError: UndefVarError: Ic not defined

## Visualize results of 1 step prediction

In [11]:
# create figure 
_, ax = plt.subplots(ncols=2, figsize=(15,5))

# plot true evolution of frequency component # add offset because prediction is only performed for second sample forwards
ax[1].plot(t[2:end], spec_select[2:end], label="true"), ax[2].plot(t[2:end], spec_select[2:end], label="true") 

# plot prediction of frequency component
ax[1].plot(t[2:end], μ_pred[1:end-1], label="prediction"), ax[2].plot(t[2:end], μ_pred[1:end-1], label="prediction")

# change visualization
ax[1].legend(), ax[1].grid(), ax[1].set_xlabel("time [sec]"), ax[1].set_ylabel("frequency component"), ax[1].set_title("entire evolution of frequency component"), ax[1].set_xlim(0, 3)
ax[2].legend(), ax[2].grid(), ax[2].set_xlabel("time [sec]"), ax[2].set_ylabel("frequency component"), ax[2].set_title("part of evolution of frequency component")
ax[2].set_xlim(2.1, 2.11), ax[2].set_ylim(-1.5,1.5), ax[1].set_ylim(-2.5, 2.5)
;

UndefVarError: UndefVarError: plt not defined

In [12]:
# loop through auto-regressive coefficients
for ki = 1:AR_order
    
    # plot auto-regressive coefficients
    plt.plot(t, [μ_θ_tot[k][ki] for k=1:iterations:length(μ_θ_tot)])
    
end

# change visualization
plt.grid(), plt.ylim(-1, 1), plt.xlim(0, 3), plt.xlabel("time [sec]"), plt.ylabel("auto-regressive coefficients θ"), plt.title("Evolution of auto-regressive coefficients")
;

UndefVarError: UndefVarError: AR_order not defined

In [13]:
# specify range of normalized frequency
theta = 0:pi/100:pi

# approximate variance of process noise by mode
pn_var = 1/((current_a_γ-1)/current_b_γ) 

# calculate analytical power spectral density of the evolution of the spectral coefficients
psd = pn_var ./(abs.([sum(-append!([-1.], μ_θ_tot[end]).*exp.(-collect(0:AR_order)*1im.*th)) for th in theta]).^2)

# create figure
_, ax = plt.subplots(ncols=3, figsize=(15,4))

# plot analytical solution of PSD
ax[1].plot(theta, psd)

# plot prediction of PSD
ax[2].plot(FFTW.fftfreq(length(μ_pred[1:end-1]), fs_desired), (abs.(FFTW.fft(μ_pred[1:end-1])).^2)./length(μ_pred[1:end-1]))

# plot FFT solution of PSD
ax[3].plot(FFTW.fftfreq(length(spec_select), fs_desired), (abs.(FFTW.fft(spec_select)).^2)./length(spec_select))

# change visualisation
ax[1].grid(), ax[2].grid(), ax[3].grid(), ax[1].set_xlabel("relative frequency θ [rad]"), ax[2].set_xlabel("frequency [Hz]"), ax[3].set_xlabel("frequency [Hz]"), ax[1].set_ylabel("power spectral density"), ax[2].set_ylabel("power spectral density"), ax[3].set_ylabel("power spectral density"), ax[1].set_title("Found analytical solution"), ax[2].set_title("Found solution"), ax[3].set_title("True solution"), ax[1].set_xlim(0, pi), ax[2].set_xlim(0, fs_desired/2), ax[3].set_xlim(0, fs_desired/2)
;

UndefVarError: UndefVarError: μ_θ_tot not defined

In [14]:
# specify range of normalized frequency
theta = 0:pi/100:pi

# approximate variance of process noise by mode
pn_var = 1/((current_a_γ-1)/current_b_γ) 

# calculate analytical power spectral density of the evolution of the spectral coefficients
psd = pn_var ./(abs.([sum(-append!([-1.], μ_θ_tot[end]).*exp.(-collect(0:AR_order)*1im.*th)) for th in theta]).^2)

# create figure
_, ax = plt.subplots(ncols=3, figsize=(15,4))

# plot analytical solution of PSD
ax[1].plot(theta, 10*log10.(psd))

# plot prediction of PSD
ax[2].plot(FFTW.fftfreq(length(μ_pred[1:end-1]), fs_desired), 10*log10.((abs.(FFTW.fft(μ_pred[1:end-1])).^2)./length(μ_pred[1:end-1])))

# plot FFT solution of PSD
ax[3].plot(FFTW.fftfreq(length(spec_select), fs_desired), 10*log10.((abs.(FFTW.fft(spec_select)).^2)./length(spec_select)))


# change visualisation
ax[1].grid(), ax[2].grid(), ax[3].grid(), ax[1].set_xlabel("relative frequency θ [rad]"), ax[2].set_xlabel("frequency [Hz]"), ax[3].set_xlabel("frequency [Hz]"), ax[1].set_ylabel("power spectral density [dB]"), ax[2].set_ylabel("unnormalized power spectral density [dB]"), ax[3].set_ylabel("unnormalized power spectral density [dB]"), ax[1].set_title("Found analytical solution"), ax[2].set_title("Found solution"), ax[3].set_title("True solution"), ax[1].set_xlim(0, pi), ax[2].set_xlim(0, fs_desired/2), ax[3].set_xlim(0, fs_desired/2)
ax[1].set_ylim(ax[3].get_ylim()[1], ax[3].get_ylim()[2])
ax[1].set_ylim(-80, 20), ax[2].set_ylim(-80,20), ax[3].set_ylim(-80,20)
;

UndefVarError: UndefVarError: μ_θ_tot not defined